In [ ]:
#load packages for scraping
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys


#load packages for data manipulation and storage
import numpy as np
import pandas as pd


import sys
import time
import argparse
from random import randint

from tqdm import tqdm

import re
from random import randint
from random import uniform

from sqlalchemy import create_engine

from collections import defaultdict

In [ ]:
def save_data(fname, df, database_filename, if_exists='append'):
    engine = create_engine('sqlite:///'+ database_filename)
    df.to_sql(fname, engine, index=False, if_exists=if_exists)
    
def load_data(fname, database_filename):
    engine = create_engine('sqlite:///' + database_filename)
    df = pd.read_sql_table(fname, con=engine)
    return df

In [ ]:
job_title = 'data scientist'
job_loc = 'california'

In [ ]:
start_url = 'https://www.indeed.com'
print('initiating webdriver...')
options = Options()
# if headless is True:
#     options.add_argument('--headless')
driver = webdriver.Firefox(firefox_options = options)
print('launch webdriver...')
driver.get(start_url)
time.sleep(5)

job = driver.find_element_by_name('q')
print(f'Job title entered: {job_title}')
job.send_keys(job_title) # enter job title
where = driver.find_element_by_name('l')
time.sleep(2)
for i in range(30):
    where.send_keys(Keys.BACKSPACE)
where.clear()
print(f'Job Location entered: {job_loc}')
where.send_keys(job_loc) # enter location
driver.find_element_by_tag_name('button').click() # click 'Find Jobs'
time.sleep(5)

In [ ]:
job_url = defaultdict(list)

print('Retrieving job urls...')

while len(job_url['urls']) < 500:
    post = driver.find_elements_by_class_name('jobsearch-SerpJobCard.row.result.clickcard')
    for p in post:
        job_url['urls'].append(p.find_element_by_tag_name('a').get_attribute('href'))
    time.sleep(randint(1,5))

    try:
        close = driver.find_element_by_id('popover-close-link')
        close.click()
    except:
        next_button = driver.find_elements_by_class_name('np')
        if len(next_button) == 2:
            next_button[1].click()
        else:
            next_button[0].click()

    if len(job_url['urls']) % 100 == 0:
        print(f"number of jobs fetched: {len(job_url['urls'])}")
url_df = pd.DataFrame.from_dict(job_url)
save_data('urls_DS', url_df, 'IndeedJob', if_exists='replace')
print(f'number of jobs fetched: {len(job_url["urls"])}')

In [ ]:
urls = load_data('urls_DS', 'IndeedJob')
urls = urls.urls.values.tolist()
print(f'Number of urls: {len(urls)}')

In [ ]:
df_dict = defaultdict(list)
error_list = []
print('Retrieving job infomation...')

In [ ]:
for url in tqdm(urls):
    driver.get(url)
    time.sleep(randint(2,10))

    try:
        job_id = driver.find_elements_by_tag_name('meta')[5].get_attribute('content').split('jk=')[1]
        job_title = driver.find_element_by_tag_name('h3').text
        company_info = driver.find_element_by_class_name('jobsearch-InlineCompanyRating.icl-u-xs-mt--xs.jobsearch-DesktopStickyContainer-companyrating').find_elements_by_tag_name('div')
        company = company_info[0].text
        location = company_info[-1].text
        jd = driver.find_element_by_class_name('jobsearch-JobComponent-description.icl-u-xs-mt--md').text
        time_post = driver.find_element_by_class_name('jobsearch-JobMetadataFooter').text.split('-')[1].strip()
        df_dict['id'].append(job_id)
        df_dict['title'].append(job_title)
        df_dict['company'].append(company)
        df_dict['location'].append(location)
        df_dict['description'].append(jd)
        df_dict['time_post'].append(time_post)
    except:
        error_list.append(url)
        next
    
    if len(df_dict['title']) % 10 == 0:
        df = pd.DataFrame.from_dict(df_dict)
        save_data('listingDataScientist', df, 'IndeedJob')
        df_dict = defaultdict(list)

In [ ]:
df_dict

In [ ]:
print('Saving...')
df = pd.DataFrame.from_dict(df_dict)
save_data('listingDataAnalyst', df, 'IndeedJob')

In [ ]:
driver.close()
print('Done')